<a href="https://colab.research.google.com/github/jurijssalijevs/EdgeVision-Advanced-Mobile-ALPR-System/blob/main/EdgeVision_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Python version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0) 
Pytorch version: 2.0.1+cu118 


In [ ]:
!nvidia-smi

Fri Jul 14 17:29:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving model.pt to model.pt


In [ ]:
import torch

model = torch.hub.load('WongKinYiu/yolov7:u6', 'custom', '/content/model.pt')  # or yolov5m, yolov5l, yolov5x, custom

In [ ]:
!pip install easyocr

In [ ]:
import easyocr

reader = easyocr.Reader(['en'])

In [ ]:
!git clone https://github.com/JaidedAI/EasyOCR.git

In [ ]:
%cd /content/EasyOCR
!pip install -r requirements.txt

In [ ]:
%cd /content/EasyOCR
!pwd

In [ ]:
import shutil
from os import listdir
from os.path import isfile, join

path = "/content"
files = [f for f in listdir(path) if isfile(join(path, f))]

for f in files:
  shutil.copy(join(path, f), join("/content/EasyOCR/easyocr/model", f"custom_{f}"))

In [ ]:
!pip install python-bidi


In [ ]:
from easyocr import easyocr

#reader = easyocr.Reader(['en', 'ru'], user_network_directory="/content/EasyOCR/easyocr/model", model_storage_directory="/content/EasyOCR/easyocr/model", recog_network='custom_model')
reader = easyocr.Reader(['en'])

In [ ]:
# show an image
import PIL
from PIL import ImageDraw
im = PIL.Image.open("/content/download.png")
im

In [ ]:
threshold = 0.5
model.conf = threshold
yolo_results_json = model(im).pandas().xyxy[0]
df = model(im).pandas()
print(df.xyxy[0])

In [ ]:
width, height = im.size

In [ ]:
yolo_results_json

In [ ]:
import numpy as np
from PIL import ImageFont

# Create an ImageDraw object
draw = ImageDraw.Draw(im)

# Specify font
# font = ImageFont.truetype("/content/drive/MyDrive/font/arial/arial.ttf", 36)

result = yolo_results_json
x1, y1, x2, y2 = result['xmin'], result['ymin'], result['xmax'], result['ymax']
plate_crop = im.crop((x1, y1, x2, y2))

draw.rectangle(((x1, y1), (x2, y2)), outline ="red")
ocr_result = reader.readtext(np.asarray(plate_crop))
if (len(ocr_result)) > 0:
  for item in ocr_result:
    print(item[1])
    # Draw text on the image
    draw.text((10, 10), item[1], fill=(255, 255, 255))
im